#### Download reference genome in FASTA from RefSeq

In [14]:
datasets download genome accession --include genome 'GCF_000017985.1'

Downloading: ncbi_dataset.zip    2.01kB 13.7MB/s
Downloading: ncbi_dataset.zip    2.01kB 13.7MB/s
Downloading: ncbi_dataset.zip    2.13kB 127kB/s
Downloading: ncbi_dataset.zip    32.8kB 1.09MB/s
Downloading: ncbi_dataset.zip    164kB 3.95MB/s
Downloading: ncbi_dataset.zip    328kB 6.18MB/s
Downloading: ncbi_dataset.zip    492kB 7.88MB/s
Downloading: ncbi_dataset.zip    721kB 9.74MB/s
Downloading: ncbi_dataset.zip    885kB 10.6MB/s
Downloading: ncbi_dataset.zip    1.08MB 11.7MB/s
Downloading: ncbi_dataset.zip    1.28MB 12.2MB/s
Downloading: ncbi_dataset.zip    1.38MB done


In [1]:
unzip ncbi_dataset.zip && rm ncbi_dataset.zip

Archive:  ncbi_dataset.zip
  inflating: README.md               
  inflating: ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: ncbi_dataset/data/GCF_000017985.1/GCF_000017985.1_ASM1798v1_genomic.fna  
  inflating: ncbi_dataset/data/dataset_catalog.json  


#### Download sequence reads in FASTQ from SRA

In [3]:
prefetch SRR030257


2022-11-01T21:36:21 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2022-11-01T21:36:21 prefetch.3.0.0: 1) Downloading 'SRR030257'...
2022-11-01T21:36:21 prefetch.3.0.0: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2022-11-01T21:36:21 prefetch.3.0.0:  Downloading via HTTPS...
2022-11-01T21:36:27 prefetch.3.0.0:  HTTPS download succeed
2022-11-01T21:36:28 prefetch.3.0.0:  'SRR030257' is valid
2022-11-01T21:36:28 prefetch.3.0.0: 1) 'SRR030257' was downloaded successfully


In [4]:
# confirm location of the data
srapath SRR030257

/home/jovyan/GVA2022 tutorials/SRR030257/SRR030257.sra


In [10]:
fasterq-dump SRR030257

spots read      : 3,800,180
reads read      : 7,600,360
reads written   : 7,600,360


#### Checking the reads

Now reads are saved as `SRR030257_1.fastq` and `SRR030257_2.fastq`. 

Check the read count and pairedness with `seqfu count` and passing wildcard `SRR030257_*.fastq`.

In [12]:
seqfu count SRR030257_*.fastq

SRR030257_1.fastq	3800180	Paired


#### Mapping with `bowtie2`

Before aligning with `bowtie2`, create bowtie2 indexes for the reference genome. Save it at `bowtie2/GCF_000017985.1`.

In [3]:
mkdir -p bowtie2

# build index. Copy relative path to reference genome
bowtie2-build ncbi_dataset/data/GCF_000017985.1/GCF_000017985.1_ASM1798v1_genomic.fna bowtie2/GCF_000017985.1

Settings:
  Output files: "bowtie2/GCF_000017985.1.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ncbi_dataset/data/GCF_000017985.1/GCF_000017985.1_ASM1798v1_genomic.fna
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1157453
Using parameters --bmax 868090 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with t

Align with `bowtie2`. It takes 
* reference genome via `-x`
* paired reads via `-1` and `-2`

and specifies the output SAM file with `-S`.

It also uses 4 threads, and measure time spent.

In [5]:
bowtie2 -x bowtie2/GCF_000017985.1 -1 SRR030257_1.fastq -2 SRR030257_2.fastq -S bowtie2/SRR030257.sam -t -p 4  

Time loading reference: 00:00:00
Time loading forward index: 00:00:00
Time loading mirror index: 00:00:00
Multiseed full-index search: 00:03:57
3800180 reads; of these:
  3800180 (100.00%) were paired; of these:
    280647 (7.39%) aligned concordantly 0 times
    3382897 (89.02%) aligned concordantly exactly 1 time
    136636 (3.60%) aligned concordantly >1 times
    ----
    280647 pairs aligned concordantly 0 times; of these:
      35637 (12.70%) aligned discordantly 1 time
    ----
    245010 pairs aligned 0 times concordantly or discordantly; of these:
      490020 mates make up the pairs; of these:
        413060 (84.29%) aligned 0 times
        70527 (14.39%) aligned exactly 1 time
        6433 (1.31%) aligned >1 times
94.57% overall alignment rate
Time searching: 00:03:57
Overall time: 00:03:57


#### Convert from SAM to BAM

In [6]:
samtools view -b bowtie2/SRR030257.sam --threads 4 > bowtie2/SRR030257.bam